# Objetos, estructuras y tipos de dato parametrizados

La filosofía del paradigma de programación orientada objetos consiste en _encapsular_ datos (variables) y acciones (funciones) en una sola estructura de datos, esto intenta disminuir la complejidad del software por medio de la abstracción.

## Objetos

Un objeto representa en una sola entidad un conjunto de características y funcionalidades. Uno de las principales acciones de los objetos consiste en que tengan la facultad de interactuar con objetos de la misma clase. Los objetos tienen propiedades y métodos.

<img src="attachment:claseobjeto.png" width="500">

<img src="img/claseobjeto.png" width=300 height=300 />

### Clase

Para implementar un objeto es necesario definir una **clase** de objeto que contendrá una función inicial de invocación automática, el **constructor**, variables y funciones para que accione el objeto.

```python
class Fraccion:
    def __init__(self, xx, yy):  # constructor
        self.num = xx
        self.den = yy     # propiedades del objeto
    
    def mostrar(self):          # metodo
        print(self.num + "/" + self.den)

a = Fraccion(3,4)   # contruye una variable de tipo Fracccion

a.mostrar()    # invoca el metodo mostrar() del objeto a

```

### Tipos de datos en Julia

Sin embargo, el manejo de objetos no es del todo eficiente en términos de velocidad de ejecución, Julia privilegia la eficiencia sin sacrificar la abstracción. En Julia es posible crear **estructuras de datos definidas por el usuario** que empaquetan variables y de manera separada se diseñan métodos que operan sobe estas estructuras por medio del **despacho múltiple**.   

<img src="img/juliaobjeto.png" width=300 height=300 />

### Definición del tipo de dato Vector2D

Es muy común la necesidad de cierta forma de comportamiento que no es capturada por los tipos de datos básicos (_built-in types_), por tanto es necesario crear un nuevo tipo de dato con ciertas funacionalidades.

Un tipo de **dato definido por el usuario** ([_composite types_](https://docs.julialang.org/en/v1/manual/types/#Composite-Types)) es una colección de datos que está fundamentado en los tipos de datos básicos (`Int, Float64, ... ` ) para los cuales está condificada su definición en Julia.

estos tipos no tienen sus propios métodos (funciones internas del tipo).
Los métodos son definidos de forma separada y están caracterizados por _todos_ y cada uno de los tipos de sus argumentos, esto es conocido como **despacho múltiple** (_multiple dispatch_). Nos referiremos a _despacho_ como el proceso de elegir la "versión" adecuada para que ejecute una función.

Vamos a definir el tipo **Vector2D**


Utilizamos `struct` para empaquetar en una estrucutra variables:

In [ ]:
struct Vector2D
    x
    y
end

`struct`define por defecto un tipo de [dato inmutable](https://docs.julialang.org/en/v1/base/base/#struct).

Una instancia de este tipo de dato no puede ser modificada después de su construcción. En cambio, usar un `mutable struct` declara un tipo cuyas instancias pueden ser modificadas. Tipos inmutables evitan manipular individualmente los elementos del objeto, tipos mutables permiten manipulaciones.

Los datos almacenados en objetos inmutables están **colocados consecutivamente en memoria** -en lugar de estar en una caja- por lo que no hay punteros al objeto, esto permite mayor velocidad de acceso al objeto; el objeto está almacenado en una forma empaquetada eficiente.

In [ ]:
a = Vector2D(2,4)  # crea una varible del tipo struct

In [ ]:
b = Vector2D(2.5, 5.5)

In [ ]:
c = Vector2D("Hola", "Crayola")

Esta primera aproximación es funcional pero no se recomienda. La razón de peso, es que Julia **tendrá que inferir** en cada ejecución el tipo de dato de las variables de la estructura y esto se traducirá en un **código lento**.

Redifiniremos `Vector2D` y especificaremos el tipo de dato de sus componentes
> Debido a que `struct` crea tipos de datos inmutables, será necesario reiniciar el kernel para redefinir la estructura

In [ ]:
struct Vector2D
    x::Float64
    y::Float64
end

In [ ]:
typeof(Vector2D)

In [ ]:
methods(Vector2D)

In [ ]:
v = Vector2D(3,4)

In [ ]:
typeof(v)

In [ ]:
v1 = Vector2D(3., 2.)

In [ ]:
v2 = Vector2D(3., 2)

In [ ]:
v3 = Vector2D(3, 2.)

In [ ]:
v4 = Vector2D(1 + 3im, 2 + im)

Es posible obtener los campos de las instancias del tipo `Vector2D` escribiendo `instacia. + <TAB>` o bien por medio

In [ ]:
fieldnames(Vector2D)

In [ ]:
v.x

In [ ]:
v.y

También podemos acceder al campo con cualquiera de los siguientes funciones:

In [ ]:
getfield(v, :x)   # por nombre

In [ ]:
getfield(v, 1)   #por orden numérico

La definición de un método que ofrezca funcionalidades a variables del tipo definido por el usuario se realiza de manera equivalente a la definición de una función genérica. Por ejemplo el operador `+` lo haremos operar sobre dos variables de tipo `Vector2D`:

In [ ]:
import Base.+

In [ ]:
methods(+)

In [ ]:
+(v::Vector2D, w::Vector2D) = Vector2D(v.x + w.x, w.y + w.y)

In [ ]:
v1 + v2

In [ ]:
v1 + v2 + v3

Julia identifica la nueva definición del método asociado al operador `+` para que funcione con objetos de tipo `Vector2D`, esto lo entenderemos como el despacho múltiple. 

Hemos creado un **objeto eficiente** -aún lo puede ser más-

## Tipos de datos parametrizados

Es intuitiva la necesidad de construir un tipo `Vector2D` que se ajuste solo a datos de tipo numérico y al mismo tiempo pueda contener: enteros, flotantes, racionales, flotantes grandes, etc. Para lograr esta funcionalidad es necesario **parametrizar** el tipo de dato creado.

Para indicarle a Julia que un tipo de dato es **parametrizado** hacemos:

In [15]:
struct Vector2D{T}
    x::T
    y::T
end

`T` es el parámetro que parametriza a la estructura `Vector2D`, hemos creado con es una especie de _template_.
La parametrización es un concepto muy poderoso que se combina con el despacho múltiple. 

In [16]:
methods(Vector2D)

# 1 method for type constructor:
[1] (::Type{Vector2D})(x::T, y::T) where T in Main at In[15]:2

In [17]:
a = Vector2D(3,5)

Vector2D{Int64}(3, 5)

In [18]:
a

Vector2D{Int64}(3, 5)

In [19]:
b = Vector2D(3.0, 5.0)

Vector2D{Float64}(3.0, 5.0)

In [20]:
b

Vector2D{Float64}(3.0, 5.0)

In [21]:
Vector2D("Hola", "Crayola")

Vector2D{String}("Hola", "Crayola")

Sin embargo aún persiste el problema, se requiere forzar el tipo `Vector2D` para que sólo contenga valores numéricos. Es necesario entonces _acotar_ el tipo de dato que puede recibir el parámetro.

Para lograr lo anterior, se indica que el parámetro `T` puede tomar cualquier subtipo que deriva de un supertipo abstracto numérico, por ejemplo:

In [5]:
struct Vector2D{T <: Real}
    x::T
    y::T
end

La instrucción `T <: R` indica que el parametro `T` es un subtipo del tipo "padre" `Real`

In [ ]:
a = Vector2D("Hola", "Crayola")

In [3]:
v = Vector2D(3, 5)

Vector2D{Int64}(3, 5)

In [4]:
w = Vector2D(3.4, 5.6)

Vector2D{Float64}(3.4, 5.6)

In [5]:
z = Vector2D(3//4, 5//7)

Vector2D{Rational{Int64}}(3//4, 5//7)

Ahora modifiquemos el método `show` de tal forma que funcione para el tipo de datos `Vector2D` parametrizado, será necesario entonces especificar la derivación del parámetro `T`. 

In [6]:
import Base.show

In [7]:
show(io::IO, v::Vector2D{T}) where {T<:Real} = print(io, "[$(v.x), $(v.y)]")

show (generic function with 242 methods)

In [8]:
z

[3//4, 5//7]

In [9]:
mi_vector = Vector2D(3,3)

[3, 3]

### Constructores del tipo Vector2D

In [10]:
mi_vector = Vector2D(3)

LoadError: MethodError: no method matching Vector2D(::Int64)
Closest candidates are:
  Vector2D(::T, !Matched::T) where T<:Real at In[1]:2

Ahora se desea crear una variable de tipo `Vector2D` de tal manera que solo reciba un argumento y lo asigne por defecto a ambas variables internas, necesitamos definir un **constructor externo**.

Podemos crear un constructor que estará fuera de la definición del tipo (_outer constructors_), estos constructores proporcionan otra forma de construir una variable del tipo `struct`.

In [11]:
Vector2D(x::T) where {T <: Real} = Vector2D(x,x)

Vector2D

In [12]:
methods(Vector2D)

# 2 methods for type constructor:
[1] (::Type{Vector2D})(x::T) where T<:Real in Main at In[11]:1
[2] (::Type{Vector2D})(x::T, y::T) where T<:Real in Main at In[1]:2

In [13]:
vv = Vector2D(3)

[3, 3]

In [14]:
zz = Vector2D(4.)

[4.0, 4.0]

Cabe hacer notar que hasta ahora no podemos crear varibles de tipo `Vector2D` que reciba argumentos con tipo de dato diferentes.

In [15]:
ww = Vector2D(2., 4)

LoadError: MethodError: no method matching Vector2D(::Float64, ::Int64)
Closest candidates are:
  Vector2D(::T) where T<:Real at In[11]:1
  Vector2D(::T, !Matched::T) where T<:Real at In[1]:2

Julia nos permite agregar parámetros adicionales a la estructura `Vector2D`

In [ ]:
struct Vector2D{T1,T2 <: Real}
    x::T1
    y::T2
end

In [ ]:
a = Vector2D(3., 5)

In [ ]:
b = Vector2D(3.5, 5.0)

In [ ]:
c = Vector2D(3, 3)